<a href="https://colab.research.google.com/github/emielrobben/Capstone_AUC/blob/master/DAIS_Ass3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/MahdiTheGreat/AItools.git
%cd AItools
%ls

Cloning into 'AItools'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 67.15 KiB | 1.32 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/AItools
Cities.zip  DAIS_Ass3.ipynb  README.md


In [3]:
!unzip "Cities.zip" -d "DAIS_Ass3"
%ls

Archive:  Cities.zip
  inflating: DAIS_Ass3/Chengdu_labeled.csv  
  inflating: DAIS_Ass3/Shenyang_labeled.csv  
  inflating: DAIS_Ass3/Beijing_labeled.csv  
  inflating: DAIS_Ass3/Guangzhou_labeled.csv  
  inflating: DAIS_Ass3/Shanghai_labeled.csv  
Cities.zip  DAIS_Ass3/  DAIS_Ass3.ipynb  README.md


In [5]:
import os

# Check the extracted folder structure
print(os.listdir("DAIS_Ass3"))


['Guangzhou_labeled.csv', 'Chengdu_labeled.csv', 'Shanghai_labeled.csv', 'Shenyang_labeled.csv', 'Beijing_labeled.csv']


In [6]:
import pandas as pd
import os

# Path to the extracted data folder
data_folder = "DAIS_Ass3"

# Corrected filenames
city_files = {
    "Beijing": "Beijing_labeled.csv",
    "Shenyang": "Shenyang_labeled.csv",
    "Guangzhou": "Guangzhou_labeled.csv",
    "Shanghai": "Shanghai_labeled.csv"
}

# Load data
city_data = {city: pd.read_csv(os.path.join(data_folder, filename)) for city, filename in city_files.items()}

# Extract relevant features and target variable
feature_columns = [col for col in city_data["Beijing"].columns if col != "PM_HIGH"]

# Create training and test sets
X_train = pd.concat([city_data["Beijing"][feature_columns], city_data["Shenyang"][feature_columns]])
Y_train = pd.concat([city_data["Beijing"]["PM_HIGH"], city_data["Shenyang"]["PM_HIGH"]])

X_test = pd.concat([city_data["Guangzhou"][feature_columns], city_data["Shanghai"][feature_columns]])
Y_test = pd.concat([city_data["Guangzhou"]["PM_HIGH"], city_data["Shanghai"]["PM_HIGH"]])

# Convert to NumPy arrays (if needed for scikit-learn)
X_train, Y_train = X_train.values, Y_train.values
X_test, Y_test = X_test.values, Y_test.values

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)


Training data shape: (2895, 10)
Test data shape: (2703, 10)


In [22]:
from sklearn.cluster import KMeans #https://scikit-learn.org/stable/modules/clustering.html#k-means
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score

class Classifier:
    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        self.centroid_labels = {}

    def fit(self, X, Y):
        # Step 1: Apply K-means clustering
        self.kmeans.fit(X)
        cluster_assignments = self.kmeans.labels_

        # Step 2: Assign a majority label to each centroid
        self.centroid_labels = {}  # Reset before training

        for cluster_id in range(self.n_clusters):
            # Get all training labels assigned to this cluster
            cluster_indices = np.where(cluster_assignments == cluster_id)[0]
            cluster_labels = Y[cluster_indices]

            # Find the majority label (0 or 1) in this cluster
            if len(cluster_labels) > 0:
                most_common_label = Counter(cluster_labels).most_common(1)[0][0]
                self.centroid_labels[cluster_id] = most_common_label

        print("Model trained with", self.n_clusters, "clusters.")


    def predict(self, X):
        """ Predicts labels for new data points based on the nearest cluster centroid. """
        # Step 1: Find the nearest cluster for each point
        cluster_assignments = self.kmeans.predict(X)

        # Step 2: Assign the label of the corresponding cluster
        predictions = np.array([self.centroid_labels[cluster] for cluster in cluster_assignments])

        return predictions

    def score(self, X, Y):
        """ Evaluates the accuracy of the classifier. """
        predictions = self.predict(X)  # Get model predictions
        accuracy = accuracy_score(Y, predictions)  # Compare with true labels
        return accuracy


# Create an instance of the classifier with a chosen number of clusters
C = Classifier(n_clusters=10)  # Tune K as needed

# Fit the model on the training data (Beijing & Shenyang)
C.fit(X_train, Y_train)

# Predict on the test set (Guangzhou & Shanghai)
predictions = C.predict(X_test)
print("Predictions:", predictions)

# Calculate and print accuracy on both training and test sets
train_score = C.score(X_train, Y_train)
test_score = C.score(X_test, Y_test)

print(f"Training Accuracy: {train_score:.4f}")
print(f"Test Accuracy: {test_score:.4f}")



Model trained with 10 clusters.
Predictions: [0. 0. 0. ... 1. 1. 1.]
Training Accuracy: 0.7472
Test Accuracy: 0.8824
